In [1]:
import requests
from kafka import KafkaProducer
import json

def send_plane_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from the api    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        
        # Publish each entry to Kafka
       
        for state in states:
            output = {}
            for key in fields.keys():
                output[key] = state[fields[key]]
            # output["icao24"] = state[0]  # Id of the plane  (ICAO24)
            # output["origin_country"] = state[2]
            # output["velocity"] = state[9]
            # output["timestamp"] = state[3]  # Timestamp de la dernière position
            # output["latitude"] = state[6]  # Latitude de l'avion
            # output["longitude"] = state[5]  # Longitude de l'avion
            producer.send(topic, value=output)
        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
        print(f"Exemple : {states[0]}")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [2]:
from datetime import datetime, timedelta
fields = {
    "icao24":0,
    "callsign":1,
    "origin_country":2,
    "velocity":9,
    "timestamp":3,
    "latitude":6,
    "longitude":5
}

api_url = "https://opensky-network.org/api/states/all?lamin=30.0&lamax=60.0&lomin=-10&lomax=10.0"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

In [9]:
import time 
for i in range(50):
    print(f"{start}-{end}")        
    send_plane_to_kafka("planes", api_url, fields)
    time.sleep(50)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-26 04:00:00-2025-01-26 05:00:00
Sent 1515 records.
Exemple : ['408120', 'VIR5C   ', 'United Kingdom', 1742915963, 1742915963, -5.5261, 52.1621, 9502.14, False, 235.09, 282, 2.6, None, 9601.2, '6025', False, 0]
2025-01-26 05:00:00-2025-01-26 06:00:00
Sent 1519 records.
Exemple : ['408120', 'VIR5C   ', 'United Kingdom', 1742916022, 1742916022, -5.7243, 52.1879, 9669.78, False, 234.99, 281.88, 2.28, None, 9753.6, '6025', False, 0]
2025-01-26 06:00:00-2025-01-26 07:00:00
Sent 1512 records.
Exemple : ['408120', 'VIR5C   ', 'United Kingdom', 1742916073, 1742916073, -5.8955, 52.2098, 9753.6, False, 237.5, 281.75, 2.28, None, 9845.04, '6025', False, 0]
2025-01-26 07:00:00-2025-01-26 08:00:00
Sent 1518 records.
Exemple : ['408120', 'VIR5C   ', 'United Kingdom', 1742916121, 1742916122, -6.0615, 52.2308, 9913.62, False, 236.79, 281.53, 4.88, None, 10005.06, '6025', False, 0]
2025-01-26 08:00:00-2025-01-26 09:00:00
Sent 1508 records.
Exemple : ['408120', 'VIR5C   ', 'United Kingdom', 17429

KeyboardInterrupt: 

In [5]:
import requests
response = requests.get("https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            origin_country = state[2]
            velocity = state[9]
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}, Velocity : {velocity}, Origin : {origin_country}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 4b1800, Timestamp: 1742915148, Latitude: 47.456, Longitude: 8.556, Velocity : 0, Origin : Switzerland
ID: 39de48, Timestamp: 1742915210, Latitude: 47.6179, Longitude: 8.0859, Velocity : 223.57, Origin : France
ID: 39de42, Timestamp: 1742915210, Latitude: 46.1809, Longitude: 5.631, Velocity : 225.77, Origin : France
ID: 3c664f, Timestamp: 1742915210, Latitude: 46.9287, Longitude: 7.5579, Velocity : 206.56, Origin : Germany
ID: 4acb04, Timestamp: 1742915210, Latitude: 42.4712, Longitude: 5.8726, Velocity : 234.27, Origin : Sweden
ID: 3c664e, Timestamp: 1742915210, Latitude: 50.1126, Longitude: 8.8155, Velocity : 109.22, Origin : Germany
ID: 4bc8c4, Timestamp: 1742915210, Latitude: 49.5963, Longitude: 9.8705, Velocity : 224.95, Origin : Turkey
ID: 3c6676, Timestamp: 1742915210, Latitude: 49.9507, Longitude: 7.2424, Velocity : 212.44, Origin : Germany
ID: 471eff, Timestamp: 1742915210, Latitude: 51.5719, Longitude: 9.6137, Velocity : 229.72, Origin : Hungary
ID: 3c65c2, Timestamp: 1742